<a href="https://colab.research.google.com/github/JayThibs/Weak-Supervised-Learning-Case-Study/blob/main/text_classifier/notebooks/03_dbpedia_14_snorkel_dataset_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel --quiet
!pip install datasets --quiet
!pip install spacy --quiet

     |████████████████████████████████| 153kB 3.9MB/s 
     |████████████████████████████████| 3.8MB 30.3MB/s 
     |████████████████████████████████| 1.8MB 52.1MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 225kB 3.9MB/s 
     |████████████████████████████████| 245kB 13.0MB/s 
     |████████████████████████████████| 112kB 17.6MB/s 


In [2]:
import pandas as pd
from snorkel.labeling import labeling_function
from datasets import load_dataset
import re

In [3]:
dbpedia_dataset = load_dataset('dbpedia_14')

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


In [4]:
dbpedia_dataset = dbpedia_dataset.rename_column("label", "labels")

In [5]:
def merge_title_with_content(example):
    example["content"] = example["title"] + " " + example["content"]
    return example

In [6]:
dbpedia_dataset = dbpedia_dataset.map(merge_title_with_content, num_proc=10)

In [7]:
dbpedia_dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 70000
    })
})

In [8]:
data = {'labels': dbpedia_dataset['train']['labels'],
        'title': dbpedia_dataset['train']['title'],
        'content': dbpedia_dataset['train']['content']}

dbpedia_train_df = pd.DataFrame(data)
dbpedia_train_df['content'] = dbpedia_train_df['content'].str.lower()
dbpedia_train_df

,labels,title,content
0,0,E. D. Abbott Ltd,e. d. abbott ltd abbott of farnham e d abbott...
1,0,Schwan-Stabilo,schwan-stabilo schwan-stabilo is a german mak...
2,0,Q-workshop,q-workshop q-workshop is a polish company loc...
3,0,Marvell Software Solutions Israel,marvell software solutions israel marvell sof...
4,0,Bergan Mercy Medical Center,bergan mercy medical center bergan mercy medi...
...,...,...,...
559995,13,Barking in Essex,barking in essex barking in essex is a black ...
559996,13,Science & Spirit,science & spirit science & spirit is a discon...
559997,13,The Blithedale Romance,the blithedale romance the blithedale romance...
559998,13,Razadarit Ayedawbon,razadarit ayedawbon razadarit ayedawbon (burm...


In [9]:
data = {'labels': dbpedia_dataset['test']['labels'],
        'title': dbpedia_dataset['test']['title'],
        'content': dbpedia_dataset['test']['content']}

dbpedia_test_df = pd.DataFrame(data)
del data
dbpedia_test_df['content'] = dbpedia_test_df['content'].str.lower()
dbpedia_test_df

,labels,title,content
0,0,TY KU,ty ku ty ku /taɪkuː/ is an american alcoholic...
1,0,Odd Lot Entertainment,odd lot entertainment oddlot entertainment fo...
2,0,Henkel,henkel henkel ag & company kgaa operates worl...
3,0,GOAT Store,goat store the goat store (games of all type ...
4,0,RagWing Aircraft Designs,ragwing aircraft designs ragwing aircraft des...
...,...,...,...
69995,13,Energy Victory,energy victory energy victory: winning the wa...
69996,13,Bestiario,bestiario bestiario is a book of 8 short stor...
69997,13,Wuthering Heights,wuthering heights wuthering heights is a nove...
69998,13,L'Indépendant,l'indépendant l'indépendant is a newspaper pu...


## A gentle introduction to LFs

Labeling functions (LFs) help users encode domain knowledge and other supervision sources programmatically.

LFs are heuristics that take as input a data point and either assign a label to it (in this case, HAM or SPAM) or abstain (don’t assign any label). Labeling functions can be noisy: they don’t have perfect accuracy and don’t have to label every data point. Moreover, different labeling functions can overlap (label the same data point) and even conflict (assign different labels to the same data point). This is expected, and we demonstrate how we deal with this later.

Because their only requirement is that they map a data point a label (or abstain), they can wrap a wide variety of forms of supervision. Examples include, but are not limited to:

    Keyword searches: looking for specific words in a sentence
    Pattern matching: looking for specific syntactical patterns
    Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
    Distant supervision: using external knowledge base
    Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

### a) Exploring the training set for initial ideas

We’ll start by looking at 20 random data points from the train set to generate some ideas for LFs.


In [10]:
Abstain = -1
Company = 0
EducationalInstitution = 1
Artist = 2
Athlete = 3
OfficeHolder = 4
MeanOfTransportation = 5
Building = 6
NaturalPlace = 7
Village = 8
Animal = 9
Plant = 10
Album = 11
Film = 12
WrittenWork = 13

In [11]:
pd.set_option('display.max_colwidth', None)

In [41]:
dbpedia_train_df[dbpedia_train_df['labels'] == 2].sample(50, random_state=42)

,labels,title,content
112823,2,Kevin Borg,kevin borg kevin james borg (born 9 june 1986 floriana malta) is a maltese pop singer who lives and works in sweden. borg rose to musical fame as the winner of the fifth series of the swedish singing competition idol in 2008. he was the third male winner and the first in the show's history to be born outside of sweden having moved there exactly one year before he was declared winner.
96298,2,Mari Kvien Brunvoll,mari kvien brunvoll mari kvien brunvoll (born 1984 in molde norway) is a norwegian folk and jazz singer and has over the last few years attracted attention at festivals in norway and internationally with her solo concept where the voice is complemented by small electronic and acoustic instruments in a sound inspired by jazz pop and folk music. she is the daughter of lawyer knut anker brunvoll (b.
108505,2,Johnny Vicious,johnny vicious johnny vicious (born john coles) is an american house dj producer and remixer.
86689,2,Steve Adey,steve adey steve adey (born in great barr birmingham england) is a musician and singer-songwriter. his music is characterised by slow tempos minimalist arrangements underpinned by piano and a rich baritone vocal. to date adey has released two albums and various eps and singles.adey released his critically well received debut album all things real in 2006 through independent record label grand harmonium records.
106893,2,Roosevelt Sykes,roosevelt sykes roosevelt sykes (january 31 1906 – july 17 1983) was an american blues musician also known as the honeydripper. he was a successful and prolific cigar-chomping blues piano player whose rollicking thundering boogie-woogie was highly influential.
116572,2,John Taglieri,john taglieri john taglieri (born august 19 1967) is an american musician singer-songwriter recording artist and record label owner. born and raised in new jersey he attended college for sound engineering in new york city and has played in various new jersey bands as a drummer lead guitarist bassist keyboardist and vocalist. he released his first solo album leap of faith in 1999 on his own record company leap dog music.
92335,2,Michiko Tanaka,michiko tanaka michiko tanaka (1909-1988) was a japanese singer and actress. she lived and worked mainly in germany where she married the actor viktor de kowa.
109591,2,Ramona Fradon,ramona fradon ramona fradon (born october 1 1926) is an american comic book and comic strip artist known for her work illustrating aquaman and brenda starr and co-creating the superhero metamorpho. her career began in 1950.
98948,2,Marusya Klimova,marusya klimova marusya klimova (first name also transliterated marusia or maroussia; russian: мару́ся кли́мова; real name tatyana nikolayevna kondratovich татьяна николаевна кондратович; born january 14 1961 in leningrad soviet union) is a russian writer and translator.klimova plays with prose and reality creating a blend of the two. vladimir sorokin victor erofeyev and ilya kabakov named klimova as one of the most prominent contemporary russian authors.
111067,2,James Chirillo,james chirillo james louis chirillo (born may 2 1953 waltham massachusetts) is an american jazz guitarist jazz banjoist composer arranger and band leader. he grew up in bellevue washington and has been a resident of teaneck new jersey since 1993.


Some things to notice for each label:

0. Company:

Words like: company, headquarter, corporate, finance, Ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, Inc., Foundation, newspaper, 

1. EducationalInstitution

Words like: University, students, bachelor, degree, school, academy, college, 

2. Artist

Words like: dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, English, American, Chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, film, screenwriter, prose, poet, career, jazz, folk, 

3. Athlete

Words like: rugby, player, football, professional, NFL, League, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, MLB, pitcher, fencer, driver, American, Canadian, English, Belgian, attended, season, champion, motorcycle

4. OfficeHolder

Words like: born, author, commentator, director, professor, leader, Governor, politician, Minister, President, died, representatives, assembly, Republic, Democratic, House of Representatives, House, election, Member of Parliament, Party, Speaker, president, lawyer,  Liberal, candidate, election, Deputy, Prime Minister, official




In [29]:


# , 'Village',
              #  'Animal', 'Plant', 'Album', 'Film', 'WrittenWork'

In [42]:
import spacy
from spacy.matcher import PhraseMatcher

# Creating keyword lists with string and split(', ').

keywords_list = ["""company, headquarter, corporate, finance, Ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, Inc., Foundation, newspaper""".split(', '),
                 """University, students, bachelor, degree, school, academy, college""".split(', '),
                 """dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, English, American, Chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, film, screenwriter, prose, poet, career, jazz, folk""".split(', '),
                 """rugby, player, football, professional, NFL, League, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, MLB, pitcher, fencer, driver, American, Canadian, English, Belgian, attended, season, champion, motorcycle""".split(', '),
                 """born, author, commentator, director, professor, leader, Governor, politician, Minister, President, died, representatives, assembly, Republic, Democratic, House of Representatives, House, election, Member of Parliament, Party, Speaker, president, lawyer, Liberal, candidate, election, Deputy, Prime Minister, official""".split(', '),
                 """ship, aircraft, Boeing, Navy, destroyer, diesel, rail, warship, Transport, submarine, trike, aerobatic, motorcar, Railway, monoplane, vessel, motorcycle, Navigation, railway, cars, airliner, Naval, whaleship, Rail, automobile, flown, ferry, wing, tailplane, car, convoy, flown, Naval, locomotive, vehicle, flagship, cruise ship, boat, convoy, automaker, battleships""".split(', '),
                 """Church, church, Center, House, house, historic, Dam, District, district, Castle, castle, Hospital, hospital, institution, museum, Victorian, farm, building, mall, restaurant, shopping mall, Centre, supermarket, built in""".split(', '),
                 """River, river, mountain, hill, hills, land, lake, Lake, km, m, Forest, forest, creek, ocean, stream, strait, gulf, peak, elevation, glacier, volcanic, corona""".split(', ')
                 ]

labels_list = ['Company', 'EducationalInstitution', 'Artist', 'Athlete', 'OfficeHolder',
               'MeanOfTransportation','Building', 'NaturalPlace']

label_keyword_dict = dict(zip(labels_list, keywords_list))
label_keyword_dict['NaturalPlace']

nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

# Only run nlp.make_doc to speed things up
for label in label_keyword_dict:
  patterns = [nlp.make_doc(text) for text in label_keyword_dict[label]]
  matcher.add(label, patterns)

doc = nlp("monika bohge monika bohge (lüdenscheid 1947) is a german writer.")
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(rule_id, span.text)

Artist writer


In [14]:
matches

[(3766102292120407359, 2, 4),
 (3766102292120407359, 7, 9),
 (3766102292120407359, 19, 22)]

In [16]:
def on_match(matcher, doc, id, matches):
    print('Matched!', matches)

matcher = PhraseMatcher(nlp.vocab)
matcher.add("OBAMA", [nlp("Barack Obama")], on_match=on_match)
matcher.add("HEALTH", [nlp("health care reform"), nlp("healthcare reform")], on_match=on_match)
doc = nlp("Barack Obama urges Congress to find courage to defend his healthcare reforms")
matches = matcher(doc)

Matched! [(7732777389095836264, 0, 2)]


In [17]:
color_patterns = [nlp(text) for text in ('red', 'green', 'yellow')]
product_patterns = [nlp(text) for text in ('boots', 'coats', 'bag')]
material_patterns = [nlp(text) for text in ('silk', 'yellow fabric')]

matcher = PhraseMatcher(nlp.vocab)
matcher.add('COLOR', None, *color_patterns)
matcher.add('PRODUCT', None, *product_patterns)
matcher.add('MATERIAL', None, *material_patterns)

In [18]:
doc = nlp("yellow fabric")
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    print(rule_id, span.text)

COLOR yellow
MATERIAL yellow fabric


In [ ]:
def on_match(matcher, doc, id, matches):
  print('Matched!', matches)

matcher = Matcher(nlp.vocab)

for label in label_keyword_dict:

  # # People, including fictional.
  matcher.add(label_keyword_dict[label][0], label_keyword_dict[label][1],
              on_match=on_match)

# Company
substance_pattern = [{'LEMMA': 'company'}]
fraser_pattern = [{'LEMMA': 'headquarter'}]
fraser_river_pattern = [{'TEXT': 'based'}, {'TEXT': 'in'}]
whale_pattern = [{'LEMMA': 'corporate'}]
construction_pattern = [{'TEXT': 'finance'}]
schedule_pattern = [{'TEXT': 'ltd.'}]
clubroot_pattern = [{'TEXT': 'airline'}]
marine_mammal_pattern = [{'TEXT': 'based'}, {'TEXT': 'out'}, {'TEXT': 'of'}]
setting_pattern = [{'LEMMA': 'firm'}]
northern_pattern = [{'TEXT': 'commerce'}]
gateway_pattern = [{'LEMMA': 'manufacturer'}]
hope_pattern = [{'LEMMA': 'factory'}]
salmon_pattern = [{'TEXT': 'founded'}]
oncorhynchus_pattern = [{'TEXT': 'Inc.'}]
hope_pattern = [{'TEXT': 'foundation'}]
salmon_pattern = [{'LEMMA': 'newspaper'}]
oncorhynchus_pattern = [{'TEXT': 'corporation'}]




In [72]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 1.6MB 6.7MB/s 
     |████████████████████████████████| 788kB 24.0MB/s 
Requirement already up-to-date: wheel in /usr/local/lib/python3.7/dist-packages (0.36.2)
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Found existing installation: setuptools 56.0.0
    Uninstalling setuptools-56.0.0:
      Successfully uninstalled setuptools-56.0.0


     |████████████████████████████████| 12.8 MB 6.6 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.4 MB/s 
     |████████████████████████████████| 456 kB 31.6 MB/s 
     |████████████████████████████████| 9.1 MB 63.5 MB/s 
     |████████████████████████████████| 113 kB 59.3 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=4fcf3e57f0cfe372510168d881e73899fa0a8fa04014210034ff70595887786c
  Stored in directory: /root/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found e

In [69]:
label_keyword_dict = {
    Abstain: ['not a word', 'still nothing'],
    
}

In [71]:
label_keyword_dict

{-1: ['not a word', 'still nothing']}

In [ ]:
from snorkel.labeling import LabelingFunction


def keyword_lookup(x, keywords, label):
    if any(word in x.text.lower() for word in keywords):
        return label
    return ABSTAIN


def make_keyword_lf(keywords, label=SPAM):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )

In [ ]:
for label in labels:
  

In [ ]:
from snorkel.preprocess.nlp import SpacyPreprocessor

# The SpacyPreprocessor parses the text in text_field and
# stores the new enriched representation in doc_field
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)


In [ ]:
from snorkel.labeling.lf.nlp import nlp_labeling_function


@nlp_labeling_function()
def has_person_nlp(x):
    """Ham comments mention specific people and are short."""
    if len(x.doc) < 20 and any([ent.label_ == "PERSON" for ent in x.doc.ents]):
        return HAM
    else:
        return ABSTAIN


In [ ]:
lfs = [
       keyword_,
       
]

In [ ]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=dbpedia)
L_test = applier.apply(df=)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=14, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [ ]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

## Filtering out unlabeled data points

As we saw earlier, some of the data points in our train set received no labels from any of our LFs. These data points convey no supervision signal and tend to hurt performance, so we filter them out before training using a built-in utility.

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)